# Installing Dependencies

In [ ]:
! pip install datasets transformers trl peft accelerate bitsandbytes auto-gptq optimum modin

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

# Importing Dependencies

In [ ]:
import torch
from datasets import load_dataset, Dataset
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model
from transformers import AutoModelForCausalLM, AutoTokenizer, GPTQConfig, TrainingArguments
from trl import SFTTrainer
import pandas as pd #modin.pandas --- Intel's libray for effeciently loading csv file from Datasets
import shutil
from peft import AutoPeftModelForCausalLM
from transformers import GenerationConfig
from transformers import AutoTokenizer
import torch

/usr/local/lib/python3.10/dist-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


/usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cpu.so: undefined symbol: cadam32bit_grad_fp32


/usr/local/lib/python3.10/dist-packages/trl/trainer/ppo_config.py:141: UserWarning: The `optimize_cuda_cache` arguement will be deprecated soon, please use `optimize_device_cache` instead.
  warnings.warn(


#Config

In [ ]:
class Config:
    MODEL_ID = "TheBloke/zephyr-7B-beta-GPTQ"
    DATASET_ID = "sriramahesh2000/summarization"
    CONTEXT_FIELD= ""
    INSTRUCTION_FIELD = "Article Text"
    TARGET_FIELD = "Summary"
    BITS = 4
    DISABLE_EXLLAMA = "auto"
    USE_EXLLAMA=False
    DEVICE_MAP = "cpu"
    USE_CACHE = False
    LORA_R = 16
    LORA_ALPHA = 16
    LORA_DROPOUT = 0.05
    BIAS = "none"
    TARGET_MODULES = ["q_proj", "v_proj"]
    TASK_TYPE = "CAUSAL_LM"
    OUTPUT_DIR = "sample4"
    BATCH_SIZE = 8
    GRAD_ACCUMULATION_STEPS = 1
    OPTIMIZER = "paged_adamw_32bit"
    LR = 2e-4
    LR_SCHEDULER = "cosine"
    LOGGING_STEPS = 50
    SAVE_STRATEGY = "epoch"
    NUM_TRAIN_EPOCHS = 1
    MAX_STEPS = 1
    FP16 = True
    PUSH_TO_HUB = True
    DATASET_TEXT_FIELD = "text"
    MAX_SEQ_LENGTH = 512
    PACKING = False

Loading Drive

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

# Zephyr Trainer
The training of the moddel is done here with appropriate parameters and required configration

In [ ]:
class ZephyrTrainer:

    def __init__(self):

        '''
        A Trainer used to train the Zephyr 7B model which beats Llama2-70b-chat model for your custom usecase

        Initialized:
        config: Parameters required for the trainer to create and process dataset, train and save model finally
        tokenizer: Tokenizer required in training loop
        '''

        self.config = Config()
        self.tokenizer = AutoTokenizer.from_pretrained(self.config.MODEL_ID)
        self.tokenizer.pad_token = self.tokenizer.eos_token

    def process_data_sample(self, example):

        '''
        Helper function to process the dataset sample by adding prompt and clean if necessary.

        Args:
        example: Data sample

        Returns:
        processed_example: Data sample post processing
        '''

        processed_example = "<|system|>\n You are a AI powered summarization Assistant. also use your own knolwedge about indian law and give the output accordingly.</s>\n<|user|>\n" + example[self.config.INSTRUCTION_FIELD] + "</s>\n<|assistant|>\n" + example[self.config.TARGET_FIELD]

        return processed_example

    def create_dataset(self):

        '''
        Downloads and processes the dataset

        Returns:
        processed_data: Training ready processed dataset
        '''

        data = load_dataset(self.config.DATASET_ID, split="train")

        print("\n====================================================================\n")
        print("\t\t\tDOWNLOADED DATASET")
        print("\n====================================================================\n")

        df = data.to_pandas()
        print(df)
        df[self.config.DATASET_TEXT_FIELD] = df[[self.config.INSTRUCTION_FIELD, self.config.TARGET_FIELD]].apply(lambda x: self.process_data_sample(x), axis=1)

        print("\n====================================================================\n")
        print("\t\t\tPROCESSED DATASET")

        print("\n====================================================================\n")
        print(df[[self.config.DATASET_TEXT_FIELD]])

        processed_data = Dataset.from_pandas(df[[self.config.DATASET_TEXT_FIELD]])
        return processed_data

    def prepare_model(self):

        '''
        Prepares model for finetuning by quantizing it and attaching lora modules to the model

        Returns:
        model - Model ready for finetuning
        peft_config - LoRA Adapter config
        '''

        bnb_config = GPTQConfig(
                                    bits=self.config.BITS,
                                    disable_exllama=self.config.DISABLE_EXLLAMA,
                                    tokenizer=self.tokenizer
                                )

        model = AutoModelForCausalLM.from_pretrained(
                                                        self.config.MODEL_ID,
                                                        quantization_config=bnb_config,
                                                        device_map=self.config.DEVICE_MAP
                                                    )

        print("\n====================================================================\n")
        print("\t\t\tDOWNLOADED MODEL")
        print("\n====================================================================\n")

        model.config.use_cache=self.config.USE_CACHE
        model.config.pretraining_tp=1
        model.gradient_checkpointing_enable()
        model = prepare_model_for_kbit_training(model)

        print("\n====================================================================\n")
        print("\t\t\tMODEL CONFIG UPDATED")
        print("\n====================================================================\n")

        peft_config = LoraConfig(
                                    r=self.config.LORA_R,
                                    lora_alpha=self.config.LORA_ALPHA,
                                    lora_dropout=self.config.LORA_DROPOUT,
                                    bias=self.config.BIAS,
                                    task_type=self.config.TASK_TYPE,
                                    target_modules=self.config.TARGET_MODULES
                                )

        model = get_peft_model(model, peft_config)

        print("\n====================================================================\n")
        print("\t\t\tPREPARED MODEL FOR FINETUNING")
        print("\n====================================================================\n")

        return model, peft_config

    def set_training_arguments(self):

        '''
        Sets the arguments for the training loop in TrainingArguments class
        '''

        training_arguments = TrainingArguments(
                                                output_dir=self.config.OUTPUT_DIR,
                                                per_device_train_batch_size=self.config.BATCH_SIZE,
                                                gradient_accumulation_steps=self.config.GRAD_ACCUMULATION_STEPS,
                                                optim=self.config.OPTIMIZER,
                                                learning_rate=self.config.LR,
                                                lr_scheduler_type=self.config.LR_SCHEDULER,
                                                save_strategy=self.config.SAVE_STRATEGY,
                                                logging_steps=self.config.LOGGING_STEPS,
                                                num_train_epochs=self.config.NUM_TRAIN_EPOCHS,
                                                max_steps=self.config.MAX_STEPS,
                                                fp16=self.config.FP16,
                                                push_to_hub=self.config.PUSH_TO_HUB
                                            )

        return training_arguments

    def train(self):

        '''
        Trains the model on the specified dataset in config
        '''

        data = self.create_dataset()
        model, peft_config = self.prepare_model()
        training_args = self.set_training_arguments()

        print("\n====================================================================\n")
        print("\t\t\tPREPARED FOR FINETUNING")
        print("\n====================================================================\n")

        trainer = SFTTrainer(
                                model=model,
                                train_dataset=data,
                                peft_config=peft_config,
                                dataset_text_field=self.config.DATASET_TEXT_FIELD,
                                args=training_args,
                                tokenizer=self.tokenizer,
                                packing=self.config.PACKING,
                                max_seq_length=self.config.MAX_SEQ_LENGTH
                            )
        trainer.train()

        print("\n====================================================================\n")
        print("\t\t\tFINETUNING COMPLETED")
        print("\n====================================================================\n")

        trainer.push_to_hub()

In [ ]:
if __name__ == "__main__":
    zephyr_trainer = ZephyrTrainer()
    zephyr_trainer.train()

/usr/local/lib/python3.10/dist-packages/pyarrow/pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):
Using `disable_exllama` is deprecated and will be removed in version 4.37. Use `use_exllama` instead and specify the version with `exllama_config`.The value of `use_exllama` will be overwritten by `disable_exllama` passed in `GPTQConfig` or stored in your config file.




			DOWNLOADED DATASET


                                           Article Text  \
0     Campbell: E-mail row 'silly fuss'\n\nEx-No 10 ...   
1     Labour plans maternity pay rise\n\nMaternity p...   
2     Howard rebuts asylum criticisms\n\nTory leader...   
3     PM apology over jailings\n\nTony Blair has apo...   
4     'Errors' doomed first Dome sale\n\nThe initial...   
...                                                 ...   
2219  Veteran Martinez wins Thai title\n\nConchita M...   
2220  Mirza makes Indian tennis history\n\nTeenager ...   
2221  Soderling wins tense Milan final\n\nFifth seed...   
2222  Roddick to face Saulnier in final\n\nAndy Rodd...   
2223  Big guns ease through in San Jose\n\nTop-seede...   

                                                Summary  
0     Mr Campbell messaged Newsnight after the progr...  
1     She said her party would boost maternity pay i...  
2     Former Tory chancellor Ken Clarke says Mr Howa...  
3     However, one of the so-call

You passed `quantization_config` to `from_pretrained` but the model you're loading already has a `quantization_config` attribute and has already quantized weights. However, loading attributes (e.g. use_exllama, exllama_config, use_cuda_fp16, max_input_length) will be overwritten with the one you passed to `from_pretrained`. The rest will be ignored.




			DOWNLOADED MODEL




			MODEL CONFIG UPDATED




			PREPARED MODEL FOR FINETUNING




ValueError: ignored

#Saving model
Move the folder to Google Drive

In [ ]:
folder_to_move = '/content/sample2'
destination_folder = '/content/drive/MyDrive/'
shutil.move(folder_to_move, destination_folder)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

#Inference
loading model localy


In [ ]:
def process_data_sample(example):

    processed_example = "<|system|>\n You are document sumarizer who is going to sumarise the content without missing any keypoints in a concise manner.truncate the input if it it beyond length you can handle.always give a complete sentence which makes sense and inform how much word you can handle.</s>\n<|user|>\n" + example["instruction"] + "</s>\n<|assistant|>\n"

    return processed_example
tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/intel hackathon/sample2")
sentence='''
appeal no. lxvi of 1949. appeal from the high court of judicature, bombay, in a reference under section 66 of the indian income tax act, 1022. k.m. munshi (n. p. nathvani, with him), for the appel lant. ' m.c. setalvad, attorney general for india (h. j. umrigar, with him), for the respondent. 1950. may 26. the judgment of the court was delivered by mehr chand mahajan j. this is an appeal against a judgment of the high court of judicature at bombay in an income tax matter and it raises the question whether munici pal property tax and urban immoveable property tax payable under the relevant bombay acts are allowable deductions under section 9 (1) (iv) of the indian income tax act. the assessee company is an investment company deriving its income from properties in the city of bombay. for the assessment year 1940 41 the net income of the assessee under the head "property" was computed by the income tax officer in the sum of rs. 6,21,764 after deducting from gross rents certain payments. the company had paid during the relevant year rs. 1,22,675 as municipal property tax and rs. 32,760 as urban property tax. deduction of these two sums was claimed under the provisions of section 9 the act. out of the first item a deduction in the sum of rs. 48,572 was allowed on the ground that this item represented tenants ' burdens paid by the assessee, otherwise the claim was disal lowed. the, appeals of the assessee to the appellate as sistant commissioner and to the income tax appellate tribu nal were unsuccessful. the tribunal, however, agreed to refer two questions of law to the high court of judicature at bombay, namely, (1) whether the municipal taxes paid by the applicant company are an allowable deduction under 555 the provisions of section 9 (1) (iv) of the indian income tax act; (2) whether the urban immoveable property taxes paid by the applicant company are an allowable deduction under section 9 (1) (iv) or under section 9 (1) (v) of the indian income tax act. a supplementary reference was made covering a third question which was not raised before us and it is not there fore necessary to refer to it. the high court answered all the three questions in the negative and hence this appeal. the question for our determination is whether the munic ipal property tax and urban immoveable property tax can be deducted as an allowance under clause (iv) of sub section (1) of section 9 of the act. the decision of the point depends firstly on the construction of the language employed in sub clause (iv) of sub section (1) of section 9 of the act, and secondly, on a finding as to the true nature and character of the liability of the owner under the relevant bombay acts for the payment of these taxes. section 9 along with the relevant clause runs thus: (1) the tax shall be payable by an assessee under the head ' income from property ' in respect of the bona fide annual value of property consisting of any buildings or lands appurtenant thereto of which he is the owner, . . subject to the following allowances, namely : (iv) where the property is subject to a mortgage or other capital charge, the amount of any interest on such mortgage or charge; where the property is subject to an annual charge not being a capital charge, the. amount of such charge; where the property is subject to a ground rent, the amount of such ground rent; and, where the property has been acquired, constructed, repaired, renewed or recon structed with borrowed capital, the amount of any interest payable on such capital; . . . " it will be seen that clause (iv) consists of four sub clauses corresponding to the four deductions allowed 556 under the clause. before the amending act of 1939, clause (iv) contained only the first, third and fourth sub clauses. under the first sub clause interest is deductible whether the amount borrowed on the security of the property was spent on the property or not
'''
inp_str = process_data_sample(
    {
        "instruction": sentence,
    }
)

inputs = tokenizer(inp_str, return_tensors="pt").to("cuda")

model = AutoPeftModelForCausalLM.from_pretrained(
    "/content/drive/MyDrive/intel hackathon/sample2",
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map="cuda")

generation_config = GenerationConfig(
    do_sample=True,
    top_k=1,
    temperature=0.1,
    max_new_tokens=256,
    pad_token_id=tokenizer.eos_token_id
)

In [ ]:
import time
st_time = time.time()
outputs = model.generate(**inputs, generation_config=generation_config)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))
print(time.time()-st_time)

<|system|>
 You are document sumarizer who is going to sumarise the content without missing any keypoints in a concise manner.truncate the input if it it beyond length you can handle.always give a complete sentence which makes sense and inform how much word you can handle. 
<|user|>

appeal no. lxvi of 1949. appeal from the high court of judicature, bombay, in a reference under section 66 of the indian income tax act, 1022. k.m. munshi (n. p. nathvani, with him), for the appel lant. ' m.c. setalvad, attorney general for india (h. j. umrigar, with him), for the respondent. 1950. may 26. the judgment of the court was delivered by mehr chand mahajan j. this is an appeal against a judgment of the high court of judicature at bombay in an income tax matter and it raises the question whether munici pal property tax and urban immoveable property tax payable under the relevant bombay acts are allowable deductions under section 9 (1) (iv) of the indian income tax act. the assessee company is an 

In [1]:
!pip freeze ? requirements.txt

absl-py==1.4.0
aiohttp==3.9.1
aiosignal==1.3.1
alabaster==0.7.13
albumentations==1.3.1
altair==4.2.2
anyio==3.7.1
appdirs==1.4.4
argon2-cffi==23.1.0
argon2-cffi-bindings==21.2.0
array-record==0.5.0
arviz==0.15.1
astropy==5.3.4
astunparse==1.6.3
async-timeout==4.0.3
atpublic==4.0
attrs==23.1.0
audioread==3.0.1
autograd==1.6.2
Babel==2.13.1
backcall==0.2.0
beautifulsoup4==4.11.2
bidict==0.22.1
bigframes==0.15.0
bleach==6.1.0
blinker==1.4
blis==0.7.11
blosc2==2.0.0
bokeh==3.3.1
bqplot==0.12.42
branca==0.7.0
build==1.0.3
CacheControl==0.13.1
cachetools==5.3.2
catalogue==2.0.10
certifi==2023.11.17
cffi==1.16.0
chardet==5.2.0
charset-normalizer==3.3.2
chex==0.1.7
click==8.1.7
click-plugins==1.1.1
cligj==0.7.2
cloudpickle==2.2.1
cmake==3.27.7
cmdstanpy==1.2.0
colorcet==3.0.1
colorlover==0.3.0
colour==0.1.5
community==1.0.0b1
confection==0.1.4
cons==0.4.6
contextlib2==21.6.0
contourpy==1.2.0
cryptography==41.0.7
cufflinks==0.17.3
cupy-cuda11x==11.0.0
cvxopt==1.3.2
cvxpy==1.3.2
cycler==0.12.1
c